In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import tensorflow as tf
import pandas as pd
import numpy as np
import string
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Embedding, Dense, Dropout, LSTM, Input, Concatenate
#from keras import Sequential
#from tensorflow.keras.layers import Embedding, Input, Dense, Flatten, Dropout, Bidirectional, LSTM
#from keras.layers import  CuDNNLSTM
#from keras.layers.merge import concatenate

In [2]:
import os
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [3]:
df = pd.read_csv('C:/Users/Donghyeok/Desktop/Work Folder/Reddit Sarcasm Detection/train-balanced-sarcasm.csv')

In [4]:
df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [7]:
train = df.dropna(subset=['comment','parent_comment'])
train

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...
...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,..."
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...
1010823,1,Perhaps this is an atheist conspiracy to make ...,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...
1010824,1,The Slavs got their own country - it is called...,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...


In [9]:
def remove_punc(comment):
    '''This function uses str methods from the string class to remove punctuation from the text.'''
    
    # replace punctuation with '' (no space)
    translator = str.maketrans('', '', string.punctuation)
    
    # return the text stripped of punctuation marks
    return comment.translate(translator)
train['comment'] = train['comment'].apply(remove_punc)
train['parent_comment'] = train['parent_comment'].apply(remove_punc)
train.head()

C:\Users\Donghyeok\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\Donghyeok\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,Yeah I get that argument At this point Id pref...
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks The wests 5 and 6 se...
2,0,They were underdogs earlier today but since Gr...,Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,Theyre favored to win
3,0,This meme isnt funny none of the new york nigg...,icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass dont kill my buzz
4,0,I could use one of those tools,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [ ]:
def remove_stopwords_and_lowercase(comment):
    '''This function lowercases words and then remove stopwords.'''
    
    comment = \
    [word.lower() for word in comment.split() if word.lower() not in stopwords_eng]
    return ' '.join(comment)
# Apply the function to each comment
train['comment'] = train['comment'].apply(remove_stopwords_and_lowercase)
train['parent_comment'] = train['parent_comment'].apply(remove_stopwords_and_lowercase)
train.head()

In [14]:
sample_df_idx = pd.read_csv('no_lemstem_index.csv').values.ravel()

In [15]:
comments = train['comment'].loc[sample_df_idx]
parent_comments = train['parent_comment'].loc[sample_df_idx]
labels = train['label'].loc[sample_df_idx]

In [ ]:
embedding_dim = 32
vocab_size = 8000
length = 100

comment_tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
comment_tokenizer.fit_on_texts(comments)

parent_tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
parent_tokenizer.fit_on_texts(parent_comments)

word_index = tokenizer.word_index

comment_train_seq = comment_tokenizer.texts_to_sequences(comments)
comment_train_pad = pad_sequences(train_seq, maxlen=length, padding='post', truncating='post')
parent_train_seq = parent_tokenizer.texts_to_sequences(parent_comments)
parent_train_pad = pad_sequences(train_seq, maxlen=length, padding='post', truncating='post')

In [ ]:
comment_train_seq = np.array(comment_train_seq)
comment_train_pad = np.array(comment_train_pad)
parent_train_seq = np.array(parent_train_seq)
parent_train_pad = np.array(parent_train_pad)
labels = np.array(labels)

In [34]:
#Architecture 2 - Multi-Input All parallel
def define_model(length, vocab_size, embedding_dim, inputClen):
    #Channel 1
    MainComment = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, embedding_dim, input_length=length)(MainComment)
    lstm1 = tf.keras.layers.LSTM(256)(embedding1)
    drop1 = Dropout(0.2)(lstm1)
    dense1 = Dense(16, activation='sigmoid')(drop1)

    #Channel 2
    ParentComment = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, embedding_dim, input_length=length)(ParentComment)
    lstm2 = tf.keras.layers.LSTM(256)(embedding2)
    drop2 = Dropout(0.2)(lstm2)
    dense2 = Dense(16, activation='sigmoid')(drop2)

#     #Channel 3
#     inputC = Input(shape=(inputClen,))
#     x = Dense((inputClen*2), activation="relu")(inputC)
#     x = Dropout(0.2)(x)
#     x = Dense(64, activation="relu")(x)
#     x = Dense(4, activation="sigmoid")(x)

    #Concatenate
    merged = Concatenate()([dense1, dense2])

    #Interpretation
    dense3 = Dense(64, activation='relu')(merged)
    dense4 = Dense(16, activation='relu')(dense3)
    outputs = Dense(1, activation='sigmoid')(dense4)
    model = tf.keras.Model(inputs=[MainComment, ParentComment], outputs=outputs)

    #Compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    #Sumarize
    print(model.summary())
    return model

In [44]:
embedding_dim = 256
vocab_size = 5000
length = 150

tokenized_comments = tokenize_text(sample_comments, vocab_size)
tokenized_parent_comments = tokenize_text(sample_parent_comments, vocab_size)

mask = np.random.rand(len(sample_df)) < 0.8
train_data = sample_comments[mask]
test_data = sample_comments[~mask]

train_data2 = sample_parent_comments[mask]
test_data2 = sample_parent_comments[~mask]

train_data3 = sample_df.iloc[:,3:7][mask]
test_data3 = sample_df.iloc[:,3:7][~mask]

train_labels = sample_labels[mask]
test_labels = sample_labels[~mask]


train_padded1 = pad_text(tokenized_comments, train_data, length)
test_padded1 = pad_text(tokenized_comments, test_data, length)
train_padded2 = pad_text(tokenized_parent_comments, train_data2, length)
test_padded2 = pad_text(tokenized_parent_comments, test_data2, length)

train_padded1 = np.array(train_padded1)
test_padded1 = np.array(test_padded1)
train_padded2 = np.array(train_padded2)
test_padded2 = np.array(test_padded2)
print(train_padded1.shape)
print(test_padded1.shape)
print(train_padded2.shape)
print(test_padded2.shape)

#labels processing
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
print(train_labels.shape)
print(test_labels.shape)

(8089, 150)
(2018, 150)
(8089, 150)
(2018, 150)
(8089,)
(2018,)


In [45]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [46]:
def process_attributes(train, test):
    
    cs = MinMaxScaler()
    train_con = cs.fit_transform(train['score'])
    print(train_con.shape)
    test_con = cs.transform(test[continuous])
    
    train_cat = pd.get_dummies(train_data3["subreddit"])
    print(train_cat.shape)
    test_cat = pd.get_dummies(test_data3["subreddit"])
    
    trainX = np.hstack([train_cat, train_con])
    testX = np.hstack([test_cat, test_con])
    
    return (trainX, testX)

In [47]:
train_input3, test_input3 = process_attributes(train_data3, test_data3)

(8089, 3)
(8089, 1571)


In [48]:
print(train_input3.shape, test_input3.shape)


(8089, 1574) (2018, 731)


In [49]:
train_input3.shape[1]

1574

In [ ]:
model = define_model(length, vocab_size, embedding_dim, train_input3.shape[1])

In [ ]:
num_epochs = 10
batch_size = 200

In [ ]:
lstm = model.fit([train_padded1, train_padded2, train_input3], train_labels, epochs=num_epochs, batch_size=batch_size, validation_split=0.2, verbose=True)

In [ ]:
plt.plot(lstm.history['loss'])
plt.plot(lstm.history['val_loss'])
plt.xlabel("Epochs")
plt.ylabel('loss')
plt.legend(['loss', 'val_loss'])
plt.show()
  
plt.plot(lstm.history['accuracy'])
plt.plot(lstm.history['val_accuracy'])
plt.xlabel("Epochs")
plt.ylabel('accuracy')
plt.legend(['accuracy', 'val_accuracy'])
plt.show()

In [24]:
predictions = model.predict([test_padded1,test_padded2,test_input3])
y_pred = predictions > 0.5

ValueError: Error when checking input: expected input_3 to have shape (1551,) but got array with shape (746,)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
matrix = confusion_matrix(test_labels, y_pred)

In [ ]:
df_cm = pd.DataFrame(matrix, index = ["Actual Negative", "Actual Positive"],
                      columns = ["Predicted Negative", "Predicted Positive"])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True, fmt='d', cmap="Blues")

In [ ]:
#Architecture 3 - Parallel + Stacked
#Architecture 2 - Multi-Input All parallel
def define_model(length, vocab_size):
    #Channel 1
    MainComment = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, embedding_dim, input_length=max_length)(inputs1)
    lstm1 = Bidirectional(tf.keras.layers.LSTM(64))(embedding1)
    drop1 = Dropout(0.5)(lstm1)

    #Channel 2
    ParentComment = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, embedding_dim, input_length=max_length)(ParentComment)
    lstm2 = Bidirectional(tf.keras.layers.LSTM(64))(embedding2)
    drop2 = Dropout(0.5)(lstm2)

    #Channel 3
    inputC = Input(shape=(10,))
    x = Dense(8, activation="relu")(inputC)
    x = Dense(4, activation="relu")(x)

    #Concatenate
    merged = concatenate([drop1, drop2])
    lstm3 = Bidirectional(tf.keras.layers.LSTM(64))(merged)
    drop3 = Dropout(0.5)(lstm3)
    
    #Concatenate

    #Interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)

    #Compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    #Sumarize
    print(model.summary())
    plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [ ]:
mlp = models.create_mlp(trainAttrX.shape[1], regress=False)
cnn = models.create_cnn(64, 64, 3, regress=False)
combinedInput = concatenate([mlp.output, cnn.output])
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [ ]:
model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
])
 